<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## LSTM Bot QA

### Datos
El objecto es utilizar datos disponibles del challenge ConvAI2 (Conversational Intelligence Challenge 2) de conversaciones en inglés. Se construirá un BOT para responder a preguntas del usuario (QA).\
[LINK](http://convai.io/data/)

In [1]:
!pip install --upgrade --no-cache-dir gdown --quiet

In [2]:
import re

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, SimpleRNN
from keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from keras.layers import Input

In [3]:
# Se descarga la carpeta de dataset
import os
import gdown
if os.access('data_volunteers.json', os.F_OK) is False:
    url = 'https://drive.google.com/uc?id=1awUxYwImF84MIT5-jCaYAPe2QwSgS1hN&export=download'
    output = 'data_volunteers.json'
    gdown.download(url, output, quiet=False)
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [4]:
# Dataset
import json

text_file = "data_volunteers.json"
with open(text_file) as f:
    data = json.load(f) # la variable data será un diccionario



In [5]:
# Estos son los campos disponibles en cada línea del dataset
data[0].keys()

dict_keys(['dialog', 'start_time', 'end_time', 'bot_profile', 'user_profile', 'eval_score', 'profile_match', 'participant1_id', 'participant2_id'])

A continuación se crea una función para limpiar el texto, se filtran las oraciones por longitud, y se preparan tres versiones de las oraciones (entrada, salida con <eos>, y entrada de decodificación con <sos>) para alimentar a un modelo basado en ncoder-decoder.

In [6]:
chat_in = []
chat_out = []

input_sentences = []
output_sentences = []
output_sentences_inputs = []

MAX_VOCAB_SIZE = 8000
max_len = 20

def clean_text(txt):
    txt = txt.lower()    
    txt.replace("\'d", " had")
    txt.replace("\'s", " is")
    txt.replace("\'m", " am")
    txt.replace("don't", "do not")
    txt = re.sub(r'\W+', ' ', txt)
    
    return txt

for line in data:
    for i in range(len(line['dialog'])-1):
        # vamos separando el texto en "preguntas" (chat_in)
        # y "respuestas" (chat_out)
        chat_in = clean_text(line['dialog'][i]['text'])
        chat_out = clean_text(line['dialog'][i+1]['text'])

        if len(chat_in) >= max_len or len(chat_out) >= max_len:
            continue

        input_sentence, output = chat_in, chat_out
        
        # output sentence (decoder_output) tiene <eos>
        output_sentence = output + ' <eos>'
        # output sentence input (decoder_input) tiene <sos>
        output_sentence_input = '<sos> ' + output

        input_sentences.append(input_sentence)
        output_sentences.append(output_sentence)
        output_sentences_inputs.append(output_sentence_input)

print("Cantidad de rows utilizadas:", len(input_sentences))

Cantidad de rows utilizadas: 1903


In [7]:
# Ejemplo de sentences
input_sentences[1], output_sentences[1], output_sentences_inputs[1]

('hi how are you ', 'not bad and you  <eos>', '<sos> not bad and you ')

### 2 - Preprocesamiento
Realizar el preprocesamiento necesario para obtener:
- word2idx_inputs, max_input_len
- word2idx_outputs, max_out_len, num_words_output
- encoder_input_sequences, decoder_output_sequences, decoder_targets

Se realiza el preprocesamiento transformando las oraciones de texto en secuencias numéricas para que el modelo pueda procesarlas. También se normalizan las longitudes de las secuencias haciendo uso del **padding**, para agregar ceros al inicio o al final y se calculan longitudes máximas de entrada y salida (max_input_len y max_output_len) para asegurar de que los datos estén correctamente alineados.

In [8]:
# Tokenización
tokenizer_inputs = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer_inputs.fit_on_texts(input_sentences)
input_sequences = tokenizer_inputs.texts_to_sequences(input_sentences)
word2idx_inputs = tokenizer_inputs.word_index
max_input_len = max(len(s) for s in input_sequences)

tokenizer_outputs = Tokenizer(num_words=MAX_VOCAB_SIZE, filters="")
tokenizer_outputs.fit_on_texts(output_sentences + output_sentences_inputs)
output_sequences = tokenizer_outputs.texts_to_sequences(output_sentences)
output_sequences_inputs = tokenizer_outputs.texts_to_sequences(output_sentences_inputs)
word2idx_outputs = tokenizer_outputs.word_index
max_output_len = max(len(s) for s in output_sequences)

# Se preparan los datos ajustando las secuencias a una longitud fija añadiendo ceros (al principio o al final, según correponda)
encoder_input_sequences = pad_sequences(input_sequences, maxlen=max_input_len)
decoder_input_sequences = pad_sequences(output_sequences_inputs, maxlen=max_output_len, padding='post')
decoder_output_sequences = pad_sequences(output_sequences, maxlen=max_output_len, padding='post')


In [9]:
# Para confirmar que sos y eos estén en el vocabulario y no se hayan filtrado
print("Indice de <sos>:", word2idx_outputs.get("<sos>"))
print("Indice de <eos>:", word2idx_outputs.get("<eos>"))

Indice de <sos>: 2
Indice de <eos>: 1


### 3 - Preparar los embeddings
Utilizar los embeddings de Glove o FastText para transformar los tokens de entrada en vectores

Se hace uso de la clase utilizada en los prácticos del curso para trabajar con embeddings preentrenados.

In [10]:
import logging
import os
from pathlib import Path
from io import StringIO
import pickle

class WordsEmbeddings(object):
    logger = logging.getLogger(__name__)

    def __init__(self):
        # load the embeddings
        words_embedding_pkl = Path(self.PKL_PATH)
        if not words_embedding_pkl.is_file():
            words_embedding_txt = Path(self.WORD_TO_VEC_MODEL_TXT_PATH)
            assert words_embedding_txt.is_file(), 'Words embedding not available'
            embeddings = self.convert_model_to_pickle()
        else:
            embeddings = self.load_model_from_pickle()
        self.embeddings = embeddings
        # build the vocabulary hashmap
        index = np.arange(self.embeddings.shape[0])
        # Dicctionarios para traducir de embedding a IDX de la palabra
        self.word2idx = dict(zip(self.embeddings['word'], index))
        self.idx2word = dict(zip(index, self.embeddings['word']))

    def get_words_embeddings(self, words):
        words_idxs = self.words2idxs(words)
        return self.embeddings[words_idxs]['embedding']

    def words2idxs(self, words):
        return np.array([self.word2idx.get(word, -1) for word in words])

    def idxs2words(self, idxs):
        return np.array([self.idx2word.get(idx, '-1') for idx in idxs])
    
    def load_model_from_pickle(self):
        self.logger.debug(
            'loading words embeddings from pickle {}'.format(
                self.PKL_PATH
            )
        )
        max_bytes = 2**28 - 1 # 256MB
        bytes_in = bytearray(0)
        input_size = os.path.getsize(self.PKL_PATH)
        with open(self.PKL_PATH, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        embeddings = pickle.loads(bytes_in)
        self.logger.debug('words embeddings loaded')
        return embeddings

    def convert_model_to_pickle(self):
        # create a numpy strctured array:
        # word     embedding
        # U50      np.float32[]
        # word_1   a, b, c
        # word_2   d, e, f
        # ...
        # word_n   g, h, i
        self.logger.debug(
            'converting and loading words embeddings from text file {}'.format(
                self.WORD_TO_VEC_MODEL_TXT_PATH
            )
        )
        structure = [('word', np.dtype('U' + str(self.WORD_MAX_SIZE))),
                     ('embedding', np.float32, (self.N_FEATURES,))]
        structure = np.dtype(structure)
        # load numpy array from disk using a generator
        with open(self.WORD_TO_VEC_MODEL_TXT_PATH, encoding="utf8") as words_embeddings_txt:
            embeddings_gen = (
                (line.split()[0], line.split()[1:]) for line in words_embeddings_txt
                if len(line.split()[1:]) == self.N_FEATURES
            )
            embeddings = np.fromiter(embeddings_gen, structure)
        # add a null embedding
        null_embedding = np.array(
            [('null_embedding', np.zeros((self.N_FEATURES,), dtype=np.float32))],
            dtype=structure
        )
        embeddings = np.concatenate([embeddings, null_embedding])
        # dump numpy array to disk using pickle
        max_bytes = 2**28 - 1 # # 256MB
        bytes_out = pickle.dumps(embeddings, protocol=pickle.HIGHEST_PROTOCOL)
        with open(self.PKL_PATH, 'wb') as f_out:
            for idx in range(0, len(bytes_out), max_bytes):
                f_out.write(bytes_out[idx:idx+max_bytes])
        self.logger.debug('words embeddings loaded')
        return embeddings
    
    
class GloveEmbeddings(WordsEmbeddings):
    WORD_TO_VEC_MODEL_TXT_PATH = 'glove.twitter.6B.300d.txt'
    PKL_PATH = 'gloveembedding.pkl'
    N_FEATURES = 300
    WORD_MAX_SIZE = 100

class FasttextEmbeddings(WordsEmbeddings):
    WORD_TO_VEC_MODEL_TXT_PATH = 'cc.en.300.vec'
    PKL_PATH = 'fasttext.pkl'
    N_FEATURES = 300
    WORD_MAX_SIZE = 60

En este caso se optó por utilizar los embeddings de GloVe. Se define una dimensionalidad de los embeddings en 300 y se incluye un número máximo de de palabras de 8.000 en la matriz de embeddings (según recomendación de la consigna).

In [11]:
model_embeddings = GloveEmbeddings()

In [ ]:
print('preparing embedding matrix...')
embedding_dim = model_embeddings.N_FEATURES
words_not_found = []

# Número máximo de palabras a incluir en la matriz
nb_words = min(MAX_VOCAB_SIZE, len(word2idx_inputs))  # vocab_size
embedding_matrix = np.zeros((nb_words, embedding_dim))

for word, i in word2idx_inputs.items():
    if i >= nb_words:
        continue
    embedding_vector = model_embeddings.get_words_embeddings(word)[0]
    
    # Se valida la dimensión del vector
    if embedding_vector is not None and len(embedding_vector) == embedding_dim:
        embedding_matrix[i] = embedding_vector
    else:
        # Palabras con embeddings no encontrados o dimensión incorrecta
        words_not_found.append(word)

print('number of null word embeddings:', np.sum(np.sum(embedding_matrix**2, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 752


### 4 - Entrenar el modelo
Entrenar un modelo basado en el esquema encoder-decoder utilizando los datos generados en los puntos anteriores. Utilice como referencias los ejemplos vistos en clase.

A continuación se define el modelo encoder-decoder. El codificador utiliza una capa de embedding seguida de una LSTM para procesar las entradas, generando un estado que se pasa al decodificador. El decodificador también usa una capa de embedding y una LSTM para predecir la secuencia de salida, que se pasa a una capa densa con activación softmax para obtener las probabilidades de las palabras de salida. El modelo se compila con el optimizador RMSprop y la pérdida de entropía cruzada y queda preparado para entrenarse con los datos de entrada y salida.

Para el modelo se utilizan los parámetros recomendados en la consigna:

- n_units = 128
- LSTM Dropout 0.2

In [20]:
# Modelo de entrenamiento
encoder_inputs = Input(shape=(max_input_len,))
x = Embedding(input_dim=len(word2idx_inputs) + 1, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(128, return_state=True, dropout=0.2)
_, state_h, state_c = encoder_lstm(x)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_output_len,))
decoder_embedding = Embedding(input_dim=len(word2idx_outputs) + 1, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(128, return_sequences=True, return_state=True, dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(word2idx_outputs) + 1, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Crear modelo completo
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Ajustar las dimensiones de las salidas del decodificador
decoder_output_data = np.expand_dims(decoder_output_sequences, -1)

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 7, 300)    │    225,900 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 7, 300)    │    214,500 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 128),     │    219,648 │ embedding_2[0][0] │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 7, 128),  │    219,648 │ embedding_3[0][0… │
│                     │ (None, 128),      │            │ lstm_2[0][1],     │
│                     │ (None, 128)]      │            │ lstm_2[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 7, 715)    │     92,235 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 971,931 (3.71 MB)

 Trainable params: 971,931 (3.71 MB)

 Non-trainable params: 0 (0.00 B)

Se entrena el modelo definido previamente utilizando las secuencias de entrada y salida del codificador y el decodificador. El entrenamiento se realiza en 50 épocas con un tamaño de lote de 32 y un 20% de los datos se usan para validación. Después de completar el entrenamiento, se definen dos modelos para inferencia: el modelo del codificador (encoder_model), que devuelve los estados finales de la LSTM para las secuencias de entrada, y el modelo del decodificador (decoder_model), que utiliza los estados del codificador y genera las predicciones de salida paso a paso, actualizando los estados internos del decodificador.

In [ ]:
# Se entrena el modelo
model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_output_data,
    batch_size=32,
    epochs=50,
    validation_split=0.2,
)

# Modelo para inferencia: Encoder
encoder_model = Model(encoder_inputs, encoder_states)

# Modelo para inferencia: Decoder
decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

Epoch 1/50


c:\Users\Usuario\anaconda3\envs\test\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_5']. Received: the structure of inputs=('*', '*')
  warnings.warn(


48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.4363 - loss: 4.4494 - val_accuracy: 0.6164 - val_loss: 2.3774
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6276 - loss: 2.1898 - val_accuracy: 0.6307 - val_loss: 2.2647
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6284 - loss: 1.9755 - val_accuracy: 0.6307 - val_loss: 2.1830
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6503 - loss: 1.8357 - val_accuracy: 0.6329 - val_loss: 2.1427
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6815 - loss: 1.7463 - val_accuracy: 0.6513 - val_loss: 2.0825
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7049 - loss: 1.6717 - val_accuracy: 0.6610 - val_loss: 2.0438
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7167 - loss: 1.6062 - val_accuracy: 0.6697 - val_loss: 2.0108
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7285 - loss: 1.5250 - val_accuracy: 0.6678 - val_loss: 1.

Se crea una función para generar respuestas a partir de una secuencia de entrada, utilizando el modelo para la inferencia. El algoritmo realiza lo siguiente:

- Toma la entrada (input_seq) y obtiene los estados del codificador a través de encoder_model
-  Inicializa la secuencia de salida con el token de inicio (\<sos\>), y en un bucle, predice una palabra a la vez utilizando el decodificador (decoder_model)
-  Para cada predicción, selecciona el índice de la palabra con la probabilidad más alta, lo mapea a la palabra correspondiente, y la agrega a la respuesta.
- El proceso continúa hasta que se genera el token de fin de secuencia (\<eos\>) o se alcanza el límite máximo de longitud de la respuesta.

In [16]:
# Función para generar respuestas
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs["<sos>"]
    decoded_sentence = ""

    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = None
        for word, index in word2idx_outputs.items():
            if sampled_token_index == index:
                sampled_word = word
                break
        if sampled_word == "<eos>" or len(decoded_sentence.split()) > max_output_len:
            break
        decoded_sentence += " " + sampled_word
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

### 5 - Inferencia
Experimentar el funcionamiento de su modelo.

Prueba de inferencia 1: "How are you?"

In [ ]:
sample_input_sentence_1 = "How are you?"
sample_input_seq_1 = pad_sequences(tokenizer_inputs.texts_to_sequences([sample_input_sentence_1]), maxlen=max_input_len)
decoded_response_1 = decode_sequence(sample_input_seq_1)
print(f"Pregunta: {sample_input_sentence_1}")
print(f"Respuesta: {decoded_response_1}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step


c:\Users\Usuario\anaconda3\envs\test\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_5', 'keras_tensor_11', 'keras_tensor_12']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Pregunta: How are you?
Respuesta: i m fine


La respuesta generada por el modelo en general es buena, con alguna falta. En lugar de "I'm fine", el modelo devuelve "i m fine", por lo que el modelo podría no estar manejando adecuadamente las contracciones. Sin embargo, el modelo responde de forma adecuada y mantiene el contexto de la conversación.

Prueba de inferencia 2: "Do you read?"

In [ ]:
sample_input_sentence_2 = "Do you read?"
sample_input_seq_2 = pad_sequences(tokenizer_inputs.texts_to_sequences([sample_input_sentence_2]), maxlen=max_input_len)
decoded_response_2 = decode_sequence(sample_input_seq_2)
print(f"Pregunta: {sample_input_sentence_2}")
print(f"Respuesta: {decoded_response_2}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Pregunta: Do you read?
Respuesta: i love to read


En este caso, la respuesta es totalmente coherente y adecuada en el contexto de la pregunta. La respuesta "I love to read" refleja una afirmación positiva a la pregunta. El modelo genera una respuesta fluida y correcta.

Prueba de inferencia 3: "Where are you from?"

In [ ]:
sample_input_sentence_3 = "Where are you from?"
sample_input_seq_3 = pad_sequences(tokenizer_inputs.texts_to_sequences([sample_input_sentence_3]), maxlen=max_input_len)
decoded_response_3 = decode_sequence(sample_input_seq_3)
print(f"Pregunta: {sample_input_sentence_3}")
print(f"Respuesta: {decoded_response_3}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Pregunta: Where are you from?
Respuesta: i am in the army


En este caso la respuesta generada no responde directamente a la pregunta "Where are you from?" (¿De dónde sos?). Aunque podría llegar a ser una respuesta válida dentro de un contexto específico. En lugar de especificar un lugar de origen (como "I am from the USA"), el modelo ofrece una respuesta relacionada con la ocupación, lo que indica que el modelo podría haber aprendido a asociar las preguntas sobre el lugar con una respuesta relacionada con la identidad o actividad (como "I'm in the army"). Esto puede reflejar que el modelo está correlacionando la pregunta con respuestas previamente vistas, pero sin una mayor precisión geográfica.